In [1]:
import json, time, os, gc, math
import datetime
from tqdm import tqdm 
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import keras
import keras.backend as K  
from keras.layers import *
from keras.models import Model
from keras.optimizers import Adam 
from keras.utils import to_categorical, multi_gpu_model
from keras.callbacks import *
from keras.preprocessing.sequence import pad_sequences
# from gensim.models import word2vec 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold
# from utils.opt import AdamW, RAdam
import logging

os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"
gpus = os.environ['CUDA_VISIBLE_DEVICES'].split(",")

Using TensorFlow backend.


In [1]:
# !pip install keras==2.2.4

In [3]:
!free -m
!nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:         112298        3592       91298          18       17407      108529
Swap:             0           0           0
Sun Jul 12 12:13:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           On   | 00000000:00:08.0 Off |                    0 |
| N/A   27C    P8     9W / 250W |      0MiB / 22919MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           On   | 00000000:0

In [4]:
maxlen = 150
w2v_size = 300
batch_size = 512
random_seed = 2020
drop_p = 0.20
sp_drop = 0.20
l2_rate = 1e-5
model_name = 'bilstm_attr_v1'

In [5]:
%%time

file_name = "data/log/"+datetime.date.today().strftime('%m%d')+"_{}.log".format(model_name)
def write_log(w):
    t0 = datetime.datetime.now().strftime('%H:%M:%S')
    info = "{} : {}\n".format(t0, w)
    print(info)
    with open(file_name, 'a') as f:
        f.write(info)
        f.write("-"*80+"\n")
write_log(model_name)

creative_id_emb_matrix = np.load("data/cid_weight_300d.npy")
ad_id_emb_matrix = np.load("data/adid_weight_300d.npy")
product_id_emb_matrix = np.load("data/pid_weight_300d.npy")
advertiser_id_emb_matrix = np.load("data/ader_weight_300d.npy")

train_cid = np.load("data/nn_data/train_cid.npy")
train_ader = np.load("data/nn_data/train_ader.npy")
train_adid = np.load("data/nn_data/train_adid.npy")
train_pid = np.load("data/nn_data/train_pid.npy")
train_pcat = np.load("data/nn_data/train_pcat.npy")
train_ind = np.load("data/nn_data/train_ind.npy")

# train_age = np.load("data/nn_data/train_age.npy")
train_gender = np.load("data/nn_data/train_gender.npy")
write_log("data loaded")

12:13:38 : bilstm_attr_v1

12:15:52 : data loaded

CPU times: user 36 ms, sys: 13.9 s, total: 13.9 s
Wall time: 2min 14s


In [6]:
%%time

train_cid = train_cid.astype(np.int32)
train_ader = train_ader.astype(np.int32)
train_adid = train_adid.astype(np.int32)
train_pid = train_pid.astype(np.int32)
train_pcat = train_pcat.astype(np.int32)
train_ind = train_ind.astype(np.int32)

# train_age = train_age.astype(np.int32)
train_gender = train_gender.astype(np.int32)

CPU times: user 1.9 s, sys: 4.57 s, total: 6.46 s
Wall time: 6.5 s


In [7]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:         112298       23517       51455          18       37325       88605
Swap:             0           0           0


In [10]:
splits = list(StratifiedKFold(n_splits=10, shuffle=True, random_state=1234).split(train_cid, train_gender))

In [11]:
for idx, (trn_idx, val_idx) in enumerate(splits):
    print(trn_idx, val_idx)

[      0       1       2 ... 2999995 2999997 2999999] [     17      21      30 ... 2999992 2999996 2999998]
[      0       1       2 ... 2999997 2999998 2999999] [     20      32      44 ... 2999926 2999936 2999995]
[      0       1       2 ... 2999997 2999998 2999999] [     19      22      24 ... 2999980 2999991 2999994]
[      0       1       2 ... 2999997 2999998 2999999] [      6      11      40 ... 2999949 2999952 2999955]
[      0       1       2 ... 2999996 2999997 2999998] [     12      15      29 ... 2999986 2999988 2999999]
[      2       3       4 ... 2999996 2999998 2999999] [      0       1      25 ... 2999976 2999979 2999997]
[      0       1       3 ... 2999997 2999998 2999999] [      2       5       7 ... 2999981 2999984 2999987]
[      0       1       2 ... 2999997 2999998 2999999] [      4       8      18 ... 2999954 2999961 2999985]
[      0       1       2 ... 2999997 2999998 2999999] [     10      16      26 ... 2999967 2999968 2999983]
[      0       1       2 ...

In [8]:
kern_reg = regularizers.l2(l2_rate)
class Attention(Layer):
    """多头自注意力机制
    """
    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.out_dim = nb_head * size_per_head
        super(Attention, self).__init__(**kwargs)
    def build(self, input_shape):
        super(Attention, self).build(input_shape)
        q_in_dim = input_shape[0][-1]
        k_in_dim = input_shape[1][-1]
        v_in_dim = input_shape[2][-1]
        self.q_kernel = self.add_weight(name='q_kernel',
                                        shape=(q_in_dim, self.out_dim),
                                        regularizer= kern_reg,
                                        initializer='glorot_normal')
        self.k_kernel = self.add_weight(name='k_kernel',
                                        shape=(k_in_dim, self.out_dim),
                                        regularizer= kern_reg,
                                        initializer='glorot_normal')
        self.v_kernel = self.add_weight(name='w_kernel',
                                        shape=(v_in_dim, self.out_dim),
                                        regularizer= kern_reg,
                                        initializer='glorot_normal')
    def mask(self, x, mask, mode='mul'):
        if mask is None:
            return x
        else:
            for _ in range(K.ndim(x) - K.ndim(mask)):
                mask = K.expand_dims(mask, K.ndim(mask))
            if mode == 'mul':
                return x * mask
            else:
                return x - (1 - mask) * 1e10
    def call(self, inputs):
        q, k, v = inputs[:3]
        v_mask, q_mask = None, None
        if len(inputs) > 3:
            v_mask = inputs[3]
            if len(inputs) > 4:
                q_mask = inputs[4]
        # 线性变换
        qw = K.dot(q, self.q_kernel)
        kw = K.dot(k, self.k_kernel)
        vw = K.dot(v, self.v_kernel)
        # 形状变换
        qw = K.reshape(qw, (-1, K.shape(qw)[1], self.nb_head, self.size_per_head))
        kw = K.reshape(kw, (-1, K.shape(kw)[1], self.nb_head, self.size_per_head))
        vw = K.reshape(vw, (-1, K.shape(vw)[1], self.nb_head, self.size_per_head))
        # 维度置换
        qw = K.permute_dimensions(qw, (0, 2, 1, 3))
        kw = K.permute_dimensions(kw, (0, 2, 1, 3))
        vw = K.permute_dimensions(vw, (0, 2, 1, 3))
        # Attention
        a = K.batch_dot(qw, kw, [3, 3]) / self.size_per_head**0.5
        a = K.permute_dimensions(a, (0, 3, 2, 1))
        a = self.mask(a, v_mask, 'add')
        a = K.permute_dimensions(a, (0, 3, 2, 1))
        a = K.softmax(a)
        # 完成输出
        o = K.batch_dot(a, vw, [3, 2])
        o = K.permute_dimensions(o, (0, 2, 1, 3))
        o = K.reshape(o, (-1, K.shape(o)[1], self.out_dim))
        o = self.mask(o, q_mask, 'mul')
        return o
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.out_dim)

In [12]:
kern_reg = regularizers.l2(l2_rate)
def get_model(input_len, emb_size, drop_p=0.2, sp_drop=0.2): 
    cid = Input(shape=(input_len, ), name='cid')
    adid = Input(shape=(input_len, ), name='adid')
    aderid = Input(shape=(input_len, ), name='aderid')
    prodid = Input(shape=(input_len, ), name='prodid')
    # timeid = Input(shape=(input_len, ), name='timeid')
    pcid = Input(shape=(input_len, ), name='pcid')
    inid = Input(shape=(input_len, ), name='inid')
    
#     clk_times = Input(shape=(input_len, ), name='clk_times')
    
    emb0 = Embedding(creative_id_emb_matrix.shape[0], creative_id_emb_matrix.shape[1],
                      weights=[creative_id_emb_matrix], trainable=False)(cid)
    emb1 = Embedding(ad_id_emb_matrix.shape[0], ad_id_emb_matrix.shape[1],
                      weights=[ad_id_emb_matrix], trainable=False)(adid)
    emb2 = Embedding(advertiser_id_emb_matrix.shape[0], advertiser_id_emb_matrix.shape[1],
                      weights=[advertiser_id_emb_matrix], trainable=False)(aderid)
    emb3 = Embedding(product_id_emb_matrix.shape[0], product_id_emb_matrix.shape[1],
                      weights=[product_id_emb_matrix], trainable=False)(prodid)
    
#     emb4 = Embedding(92, 18, embeddings_regularizer=kern_reg)(timeid)
    emb5 = Embedding(20, 50, embeddings_regularizer=kern_reg)(pcid)
    emb6 = Embedding(338, 100, embeddings_regularizer=kern_reg)(inid)
    
    sdrop = SpatialDropout1D(sp_drop)
    x = Concatenate()([sdrop(emb0), sdrop(emb1), sdrop(emb2), sdrop(emb3), sdrop(emb5), sdrop(emb6)])
    
    # x = Dropout(drop_p)(Bidirectional(CuDNNGRU(128, return_sequences=True))(x))
    x = Dropout(drop_p)(Bidirectional(CuDNNGRU(256, return_sequences=True))(x))
    # x2 = Dropout(drop_p)(Bidirectional(CuDNNGRU(50, return_sequences=True))(x))
    # x = Concatenate()([x1, x2])
    #mask = Lambda(lambda x: K.cast(K.greater(K.expand_dims(x, 2), 0), 'float32'))(cid)
    #x = Attention(8, 32)([x, x, x, mask])
    x = TimeDistributed(Dense(256, activation="tanh"))(x)
    x_max = Lambda(lambda x: K.max(x, axis=1), output_shape=(256,))(x)
    x_avg = Lambda(lambda x: K.mean(x, axis=1), output_shape=(256,))(x)
    
    dnn_input = Concatenate()([x_max, x_avg])
    output = Dropout(0.2)(Dense(512, activation='relu')(dnn_input))
    output = Dense(256, activation='relu')(output)
    pred = Dense(2, activation="softmax")(output)
    
    return Model(inputs=[cid, aderid, adid, prodid, pcid, inid], outputs=pred)

# 加载测试集数据

In [13]:
%%time

test_cid = np.load("data/nn_data/test_cid.npy")
test_ader = np.load("data/nn_data/test_ader.npy")
test_adid = np.load("data/nn_data/test_adid.npy")
test_pid = np.load("data/nn_data/test_pid.npy")
test_pcat = np.load("data/nn_data/test_pcat.npy")
test_ind = np.load("data/nn_data/test_ind.npy")

print(test_cid.shape, test_pid.shape, test_ind.shape)

(1000000, 150) (1000000, 150) (1000000, 150)
CPU times: user 20 ms, sys: 2.29 s, total: 2.31 s
Wall time: 23.3 s


In [14]:
train_gender = keras.utils.to_categorical(train_gender)
print(train_gender.shape)

pred_test_gender = np.zeros((test_cid.shape[0], 2))
oof_train = np.zeros((train_cid.shape[0], 2))

(3000000, 2)


In [ ]:
for idx, (trn_idx, val_idx) in enumerate(splits):
   
    write_log("fold : {}".format(idx))
    K.clear_session()

    with tf.device('/cpu:0'):
        model = get_model(maxlen, w2v_size, drop_p, sp_drop)
        # print(model.summary())

    if len(gpus)>=2:
        model = multi_gpu_model(model, gpus=len(gpus))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
    
    filepath = "save_model/gender_bigru_fold_{}.h5".format(idx+1)
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=1, min_lr=0.00005, verbose=1)
    earlystopping = EarlyStopping(monitor='val_acc', min_delta=0.000001, patience=2, verbose=1, mode='max')
    callbacks = [checkpoint, reduce_lr, earlystopping]
    
    hist = model.fit([train_cid[trn_idx], train_ader[trn_idx], train_adid[trn_idx], 
                      train_pid[trn_idx], train_pcat[trn_idx], train_ind[trn_idx]], train_gender[trn_idx], 
                     validation_data=([train_cid[val_idx], train_ader[val_idx], train_adid[val_idx],
                                        train_pid[val_idx], train_pcat[val_idx], train_ind[val_idx]], train_gender[val_idx]),
                     epochs=40, batch_size=512, callbacks=callbacks, verbose=1)
    write_log(str(hist.history))
    
    model.load_weights("save_model/gender_bigru_fold_{}.h5".format(idx+1))
    oof_train[val_idx] = model.predict([train_cid[val_idx], train_ader[val_idx], train_adid[val_idx],
                                        train_pid[val_idx], train_pcat[val_idx], train_ind[val_idx]], batch_size=1024, verbose=1)
    per_pred = model.predict([test_cid, test_ader, test_adid, test_pid, test_pcat, test_ind], 
                             batch_size=1024, verbose=1)
    
    pred_test_gender += per_pred / len(splits)
#     break

np.save("data/nn_data/oof_train_gender_bigru.npy", oof_train)
np.save("data/nn_data/pred_test_gender_bigru.npy", pred_test_gender)

12:32:20 : fold : 0












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2700000 samples, validate on 300000 samples
Epoch 1/40
2700000/2700000 [==============================] - 1064s 394us/step - loss: 0.1623 - acc: 0.9431 - val_loss: 0.1505 - val_acc: 0.9475

Epoch 00001: val_acc improved from -inf to 0.94745, saving model to save_model/gender_bigru_fold_1.h5
Epoch 2/40
2700000/2700000 [==============================] - 1021s 378us/step - loss: 0.1527 - acc: 0.9470 - val_loss: 0.1483 - val_acc: 0.9485

Epoch 00002: val_acc improved from 0.94745 to 0.94846, saving model to save_model/gender_bigru_fold_1.h5
Epoch 3/40
2700000/2700000 [==============================] - 1021s 378us/step - loss: 0.1490 - acc: 0.9483 - val_loss: 0.1478 - val_acc: 0.9486

Epoch 00003: val_acc improved from 0.94846 to 0.94861, sa

# 加载模型并测试

In [ ]:
for idx, (trn_idx, val_idx) in enumerate(splits):

    write_log("fold : {}".format(idx))
    K.clear_session()

    with tf.device('/cpu:0'):
        model = get_model(maxlen, w2v_size, drop_p, sp_drop)
        # print(model.summary())
        
    if len(gpus)>=2:
        model = multi_gpu_model(model, gpus=len(gpus))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
    
#     filepath = "best_model_bilstm_fold_{}.h5".format(idx+1)
#     checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only=True)
#     reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=1, min_lr=0.00005, verbose=1)
#     earlystopping = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=2, verbose=1, mode='max')
#     callbacks = [checkpoint, reduce_lr, earlystopping]
    
#     hist = model.fit([train_cid[trn_idx], train_ader[trn_idx], train_adid[trn_idx], 
#                       train_pid[trn_idx], train_pcat[trn_idx], train_ind[trn_idx]], train_age[trn_idx], 
#                      validation_data=([train_cid[val_idx], train_ader[val_idx], train_adid[val_idx],
#                                         train_pid[val_idx], train_pcat[val_idx], train_ind[val_idx]], train_age[val_idx]),
#                      epochs=40, batch_size=512, callbacks=callbacks, verbose=1)
#     write_log(str(hist.history))
    
    model.load_weights("best_model_gender_fold_{}.h5".format(idx+1))
    oof_train[val_idx] = model.predict([train_cid[val_idx], train_ader[val_idx], train_adid[val_idx],
                                        train_pid[val_idx], train_pcat[val_idx], train_ind[val_idx]], batch_size=1024, verbose=1)
    per_pred = model.predict([test_cid, test_ader, test_adid, test_pid, test_pcat, test_ind], 
                             batch_size=1024, verbose=1)
    
    pred_test_age += per_pred / len(splits)
# #     break

np.save("data/nn_data/oof_train.npy", oof_train)
np.save("data/nn_data/pred_test_age.npy", pred_test_age)

01:36:34 : fold : 0












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


1000000/1000000 [==============================] - 187s 187us/step
01:42:27 : fold : 1

1000000/1000000 [==============================] - 189s 189us/step
01:48:19 : fold : 2

1000000/1000000 [==============================] - 189s 189us/step
01:54:11 : fold : 3

1000000/1000000 [==============================] - 188s 188us/step
02:00:03 : fold : 4

1000000/1000000 [==============================] - 188s 188us/step
02:05:55 : fold : 5

1000000/1000000 [==============================] - 188s 188us/step
02:11:45 : fold : 6

1000000/1000000 [==============================] - 187s 187us/step
02:17:35 : fold : 7

1000000/1000000 [==============================] - 189s 189us/step
02:23:28 : fold : 8

 402432/1000000 [===========>..................] - ETA: 1:52

In [21]:
# ```PYTHON
####opt
class_num=2
weights = [1.0]*class_num

def search_weight(valid_y, raw_prob, init_weight=[1.0]*class_num, step=0.001):
    weight = init_weight.copy()
    f_best = accuracy_score(y_true=valid_y, y_pred=raw_prob.argmax(axis=1))
    flag_score = 0
    round_num = 1
    while(flag_score != f_best):
        print("round: ", round_num)
        round_num += 1
        flag_score = f_best
        for c in range(class_num):
            for n_w in range(0, 2000,10):
                num = n_w * step
                new_weight = weight.copy()
                new_weight[c] = num

                prob_df = raw_prob.copy()
                prob_df = prob_df * np.array(new_weight)

                f = accuracy_score(y_true=valid_y, y_pred=prob_df.argmax(axis=1))
                if f > f_best:
                    weight = new_weight.copy()
                    f_best = f
                    print(f)
    return weight

weight = search_weight(train_gender.argmax(-1), oof_train)
print(weight)

round:  1
0.949405
round:  2
[1.0, 1.01]


In [5]:
# oof_train = np.load("data/nn_data/oof_train_gender.npy")
pred_gender_gru = np.load("data/nn_data/pred_test_gender_bigru.npy")
pred_gender_lstm = np.load("data/nn_data/pred_test_gender_bilstm.npy")

In [6]:
pred_gender = pred_gender_gru * 0.5 + pred_gender_lstm * 0.5

In [19]:
print(accuracy_score(train_gender.argmax(-1), oof_train.argmax(-1)))
# print(accuracy_score(train_gender.argmax(-1), (weight*oof_train).argmax(-1)))

0.9494043333333333


In [16]:
# weight = np.array([0.96, 1.03, 1.0, 1.0, 1.0, 0.99, 1.0, 0.99, 1.0, 0.88])
weight = np.array([0.82, 0.97, 1.0, 1.0, 1.0, 1.0, 1.0, 0.88, 0.97, 1.01])
pred_test_age = pred_test_age * weight

In [14]:
test_pred = model.predict([test_cid, test_ader, test_pid], batch_size=512, verbose=1)

1000000/1000000 [==============================] - 171s 171us/step


In [7]:
# age = np.argmax(pred_test_age, axis=1) + 1
gender = np.argmax(pred_gender, axis=1) + 1
test_submit = pd.read_csv("data/submit/post_process_2.csv")
test_submit['predicted_age'] += 10
test_submit['predicted_gender'] = gender
#np.save(filename+".npy", pred_test_age)
test_submit.to_csv("data/submit/B_gender_10fold_gru_lstnm.csv", header=True, index=None)

In [39]:
test_submit

,user_id,predicted_age,predicted_gender
0,3000001,3,6
1,3000002,7,7
2,3000003,3,7
3,3000004,3,6
4,3000005,4,6
...,...,...,...
999995,3999996,2,6
999996,3999997,2,6
999997,3999998,2,6
999998,3999999,3,6


# 融合

In [2]:
import pandas as pd
import numpy as np

test_age_pred_zhan = np.load("data/submit/age.npy")
test_gender_pred = np.load("data/submit/gender.npy")
test_age_pred_cong = np.load("data/nn_data/pred_test_age.npy")

In [3]:
test_gender_pred = (test_gender_pred > 0.5) + 1
test_gender_pred = test_gender_pred.reshape(-1)

In [4]:
test_gender_pred[:10]

array([1, 2, 2, 1, 1, 1, 1, 1, 1, 2])

In [5]:
test_age_pred = test_age_pred_zhan * 0.5 + test_age_pred_cong * 0.5 

In [6]:
age = np.argmax(test_age_pred, axis=1) + 1
test_submit = pd.read_csv("data/submit/post_process_2.csv")
test_submit['predicted_age'] = age
test_submit['predicted_gender'] = test_gender_pred
#np.save(filename+".npy", pred_test_age)
test_submit.to_csv("data/submit/B_bilstm_10fold_ronghe.csv", header=True, index=None)

In [ ]:
4006668800 1 3 2